## 安装依赖
部分依赖 Colab 已经满足

In [1]:
%pip install pycwt TextGrid test-tube horology pyloudnorm g2p-en imageio-ffmpeg Resemblyzer pypinyin proglog pytorch-lightning pydub praat-parselmouth einops miditoolkit pytorch-ssim webrtcvad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 758 kB 15.1 MB/s 
     |████████████████████████████████| 3.1 MB 59.5 MB/s 
     |████████████████████████████████| 26.9 MB 1.3 MB/s 
     |████████████████████████████████| 15.7 MB 390 kB/s 
     |████████████████████████████████| 1.4 MB 66.8 MB/s 
     |████████████████████████████████| 701 kB 50.3 MB/s 
     |████████████████████████████████| 10.7 MB 67.8 MB/s 
     |████████████████████████████████| 66 kB 6.2 MB/s 
     |████████████████████████████████| 180 kB 73.0 MB/s 
     |████████████████████████████████| 78 kB 9.7 MB/s 
     |████████████████████████████████| 141 kB 73.0 MB/s 
     |████████████████████████████████| 419 kB 65.4 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
     |████████████████████████████████| 5.9 MB 61.9 MB/s 
     |████████████████████████████████| 51 kB 10.1 MB/s 
  Created wheel for pycwt: filename

## 从 GitHub 拉取代码

这里使用 OpenVPI 团队的 fork
https://github.com/openvpi/DiffSinger

In [1]:
%%bash
git clone https://github.com/openvpi/DiffSinger

Cloning into 'DiffSinger'...


### 拉取模型

In [2]:
%%bash
curl -o 0102_xiaoma_pe.zip -L https://github.com/MoonInTheRiver/DiffSinger/releases/download/pretrain-model/0102_xiaoma_pe.zip
unzip -o -d /content/DiffSinger/checkpoints/0102_xiaoma_pe 0102_xiaoma_pe.zip

Archive:  0102_xiaoma_pe.zip
  inflating: /content/DiffSinger/checkpoints/0102_xiaoma_pe/config.yaml  
  inflating: /content/DiffSinger/checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt  


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 34.1M  100 34.1M    0     0   103M      0 --:--:-- --:--:-- --:--:--  204M


In [3]:
%%bash
curl -o 0109_hifigan_bigpopcs_hop128.zip -L https://github.com/MoonInTheRiver/DiffSinger/releases/download/pretrain-model/0109_hifigan_bigpopcs_hop128.zip
unzip -o -d /content/DiffSinger/checkpoints/0109_hifigan_bigpopcs_hop128 0109_hifigan_bigpopcs_hop128.zip

Archive:  0109_hifigan_bigpopcs_hop128.zip
  inflating: /content/DiffSinger/checkpoints/0109_hifigan_bigpopcs_hop128/config.yaml  
  inflating: /content/DiffSinger/checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt  


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  899M  100  899M    0     0   117M      0  0:00:07  0:00:07 --:--:--  114M


In [4]:
%%bash
curl -o 0228_opencpop_ds100_rel.zip -L https://github.com/MoonInTheRiver/DiffSinger/releases/download/pretrain-model/0228_opencpop_ds100_rel.zip
unzip -o 0228_opencpop_ds100_rel.zip -x "generated_160000_/*" -d /content/DiffSinger/checkpoints/0228_opencpop_ds100_rel

Archive:  0228_opencpop_ds100_rel.zip
  inflating: /content/DiffSinger/checkpoints/0228_opencpop_ds100_rel/config.yaml  
  inflating: /content/DiffSinger/checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt  


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  380M  100  380M    0     0  92.8M      0  0:00:04  0:00:04 --:--:-- 95.0M


## 运行

### 初始化

In [5]:
import os
os.chdir("/content/DiffSinger")
os.environ["PYTHONPATH"] = "/content/DiffSinger"

import sys

sys.argv = [__name__,
            "--config",
            "usr/configs/midi/e2e/opencpop/ds100_adj_rel.yaml",
            "--exp_name",
            "0228_opencpop_ds100_rel"
            ]

from IPython.display import Audio

import inference.svs.ds_e2e as ds

### 合成样例

#### 直接输入歌词文本

以下是 DiffSinger 作者团队提供的样例，参考 [https://huggingface.co/spaces/Silentlin/DiffSinger](https://huggingface.co/spaces/Silentlin/DiffSinger)

In [6]:
inp = {
        'text': '小酒窝长睫毛AP是你最美的记号',
        'notes': 'C#4/Db4 | F#4/Gb4 | G#4/Ab4 | A#4/Bb4 F#4/Gb4 | F#4/Gb4 C#4/Db4 | C#4/Db4 | rest | C#4/Db4 | A#4/Bb4 | G#4/Ab4 | A#4/Bb4 | G#4/Ab4 | F4 | C#4/Db4',
        'notes_duration': '0.407140 | 0.376190 | 0.242180 | 0.509550 0.183420 | 0.315400 0.235020 | 0.361660 | 0.223070 | 0.377270 | 0.340550 | 0.299620 | 0.344510 | 0.283770 | 0.323390 | 0.360340',
        'input_type': 'word'
    }
target = "/content/DiffSinger/infer_out/小酒窝.wav"
ds.DiffSingerE2EInfer.example_run(inp, target=target)
Audio(filename=target)

| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
Pass word-notes check.
29 29 29
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 74.78it/s]


| save audio: /content/DiffSinger/infer_out/小酒窝.wav


In [ ]:
inp_2 = {
        'text': '你 说 你 不 SP 懂 为 何 在 这 时 牵 手 AP',
        'notes': 'D#4/Eb4 | D#4/Eb4 | D#4/Eb4 | D#4/Eb4 | rest | D#4/Eb4 | D4 | D4 | D4 | D#4/Eb4 | F4 | D#4/Eb4 | D4 | rest',
        'notes_duration': '0.113740 | 0.329060 | 0.287950 | 0.133480 | 0.150900 | 0.484730 | 0.242010 | 0.180820 | 0.343570 | 0.152050 | 0.266720 | 0.280310 | 0.633300 | 0.444590',
        'input_type': 'word'
    }
target = "/content/DiffSinger/infer_out/你说你不懂为何在这时牵手.wav"
ds.DiffSingerE2EInfer.example_run(inp_2, target=target)
Audio(filename=target)

| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
Pass word-notes check.
26 26 26
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 79.07it/s]


| save audio: /content/DiffSinger/infer_out/你说你不懂为何在这时牵手.wav


#### 手动指定音素时长

以下几条为 OpenVPI 团队提供的样例：

In [ ]:
c = {
        'text': '小酒窝长睫毛AP是你最美的记号',
        'ph_seq': 'x iao j iu w o ch ang ang j ie ie m ao AP sh i n i z ui m ei d e j i h ao',
        'note_seq': 'C#4/Db4 C#4/Db4 F#4/Gb4 F#4/Gb4 G#4/Ab4 G#4/Ab4 A#4/Bb4 A#4/Bb4 F#4/Gb4 F#4/Gb4 F#4/Gb4 C#4/Db4 C#4/Db4 C#4/Db4 rest C#4/Db4 C#4/Db4 A#4/Bb4 A#4/Bb4 G#4/Ab4 G#4/Ab4 A#4/Bb4 A#4/Bb4 G#4/Ab4 G#4/Ab4 F4 F4 C#4/Db4 C#4/Db4',
        'note_dur_seq': '0.407140 0.407140 0.376190 0.376190 0.242180 0.242180 0.509550 0.509550 0.183420 0.315400 0.315400 0.235020 0.361660 0.361660 0.223070 0.377270 0.377270 0.340550 0.340550 0.299620 0.299620 0.344510 0.344510 0.283770 0.283770 0.323390 0.323390 0.360340 0.360340',
        'is_slur_seq': '0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0',
        'ph_dur': '0.175 0.236 0.082 0.303 0.069 0.161 0.145 0.371 0.18 0.08 0.225 0.216 0.115 0.274 0.215 0.181 0.203 0.107 0.223 0.073 0.213 0.079 0.263 0.048 0.223 0.108 0.219 0.128 0.246',
        'input_type': 'phoneme'
    }  # input like Opencpop dataset.
target = "/content/DiffSinger/infer_out/小酒窝2.wav"
ds.DiffSingerE2EInfer.example_run(c, target=target)
Audio(filename=target)

| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
29 29 29 29
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 81.70it/s]


| save audio: infer_out/example_out.wav


In [7]:
# 《涟漪》前两句
c_ripples = {
  'text': '将 漂 浮 的 声 音 都 吞 没 ' + \
          'SP 描 绘 着 贝 壳 上 的 轮 廓 ',
  'ph_seq': 'j iang p iao f u d e sh eng y in d ou t un m o ' + \
            'SP m iao h ui zh e b ei k e sh ang d e l un k uo ',
  'note_seq': 'E4 E4 F#4 F#4 F#4 F#4 F#4 F#4 E4 E4 D4 D4 E4 E4 F#4 F#4 F#4 F#4 ' + \
              'rest E4 E4 F#4 F#4 F#4 F#4 F#4 F#4 E4 E4 D4 D4 A4 A4 E4 E4 F#4 F#4 ',
  'note_dur_seq': '0.36 0.36 0.3 0.3 0.36 0.36 0.24 0.24 0.36 0.36 0.3 0.3 0.51 0.51 0.33 0.33 0.705 0.705 ' + \
                  '0.225 0.27 0.27 0.33 0.33 0.315 0.315 0.255 0.255 0.3 0.3 0.36 0.36 0.57 0.57 0.27 0.27 0.705 0.705 ',
  'is_slur_seq': '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0' + \
                 '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ',
  'ph_dur': '0.165 0.18 0.105 0.195 0.105 0.255 0.045 0.195 0.105 0.255 0.045 0.255 0.045 0.465 0.135 0.195 0.105 0.6 ' + \
            '0.225 0.075 0.195 0.105 0.225 0.075 0.24 0.06 0.195 0.105 0.195 0.105 0.255 0.045 0.525 0.075 0.195 0.105 0.6 ',
  'input_type': 'phoneme'
}
target = "/content/DiffSinger/infer_out/涟漪.wav"
ds.DiffSingerE2EInfer.example_run(c_ripples, target=target)
Audio(filename=target)

| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
37 37 37 37
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 73.85it/s]


| save audio: /content/DiffSinger/infer_out/涟漪.wav


In [8]:
c_xiaoshou = {
  "text": "还 记 得 那 场 音 乐 会 的 烟 火  SP 还 记 得 那 个 凉 凉 的 深  秋  SP 还 记 得 人 潮 把 你 推 向 了 我  SP 游 乐 园 拥 挤 的 正 是 时 候  SP 一 个 夜 晚 坚 持 不 睡 的 等 候  SP 一 起 泡 温 泉 奢 侈 的 享  受  SP 有 一 次 日 记 里 愚 蠢 的 困 惑  SP 因 为 你 的 微 笑 幻 化 成 风  SP 你 大 大 的 勇 敢 保 护 着 我  SP 我 小 小 的 关 怀 喋 喋 不 休  SP 感 谢 我 们 一 起 走 了 那 么 久  SP 又 再 一 次 回 到  凉 凉 深 秋  SP 给 你 我 的 手 SP 像 温 柔 野 兽 SP 把 自 由 交 给 草 原 的 辽  阔   SP 我 们 小 手 拉 大 手 SP 一 起 郊  游 SP 今 天 别 想 太 多  SP 你 是 我 的 梦 SP 像 北 方 的 风 SP 吹 着 南 方 暖 洋 洋 的 哀  愁   SP 我 们 小 手 拉 大 手 SP 今 天 加  油 SP 向 昨 天 挥 挥  手  ",
  "ph_seq": "h ai j i d e n a ch ang y in y ve h ui d e y an h uo uo SP h ai j i d e n a g e l iang l iang d e sh en en q iu iu SP h ai j i d e r en ch ao b a n i t ui x iang l e w o o SP y ou l e y van y ong j i d e zh eng sh i sh i h ou ou SP y i g e y e w an j ian ch i b u sh ui d e d eng h ou ou SP y i q i p ao w en q van sh e ch i d e x iang iang sh ou ou SP y ou y i c i r i j i l i y v ch un d e k un h uo uo SP y in w ei n i d e w ei x iao h uan h ua ch eng f eng eng SP n i d a d a d e y ong g an b ao h u zh e w o o SP w o x iao x iao d e g uan h uai d ie d ie b u x iu iu SP g an x ie w o m en y i q i z ou l e n a m e j iu iu SP y ou z ai y i c i h ui d ao ao l iang l iang sh en q iu iu SP g ei n i w o d e sh ou SP x iang w en r ou y e sh ou SP b a z i y ou j iao g ei c ao y van d e l iao iao k uo uo uo SP w o m en x iao sh ou l a d a sh ou SP y i q i j iao iao y ou SP j in t ian b ie x iang t ai d uo uo SP n i sh i w o d e m eng SP x iang b ei f ang d e f eng SP ch ui zh e n an f ang n uan y ang y ang d e ai ai ch ou ou ou SP w o m en x iao sh ou l a d a sh ou SP j in t ian j ia ia y ou SP x iang z uo t ian h ui h ui ui sh ou ou ou",
  "note_seq": "G3 G3 G3 G3 A3 A3 C4 C4 D4 D4 E4 E4 A4 A4 G4 G4 E4 E4 D4 D4 D4 D4 C4 rest C4 C4 D4 D4 C4 C4 B3 B3 C4 C4 F4 F4 A3 A3 C4 C4 D4 D4 E4 E4 E4 D4 rest D4 D4 E4 E4 D4 D4 C#4 C#4 D4 D4 G4 G4 B3 B3 D4 D4 E4 E4 D4 D4 D4 D4 C4 rest C4 C4 D4 D4 C4 C4 B3 B3 C4 C4 F4 F4 A3 A3 C4 C4 A3 A3 A3 A3 G3 rest G3 G3 G3 G3 A3 A3 C4 C4 D4 D4 E4 E4 A4 A4 G4 G4 E4 E4 D4 D4 D4 D4 C4 rest C4 C4 D4 D4 C4 C4 B3 B3 C4 C4 F4 F4 A3 A3 C4 C4 D4 D4 E4 E4 E4 D4 rest D4 D4 E4 E4 D4 D4 C#4 C#4 D4 D4 G4 G4 B3 B3 D4 D4 E4 E4 D4 D4 D4 D4 C4 rest C4 C4 D4 D4 C4 C4 B3 B3 C4 C4 F4 F4 A3 A3 C4 C4 D4 D4 D4 D4 C4 rest E4 E4 F4 F4 E4 E4 D4 D4 E4 E4 F4 F4 E4 E4 D4 D4 E4 E4 E4 E4 F4 rest F4 F4 G4 G4 F4 F4 G4 G4 F4 F4 E4 E4 D4 D4 C4 C4 D4 D4 D4 D4 E4 rest E4 E4 E4 E4 D4 D4 C#4 C#4 E4 E4 E4 E4 D4 D4 D4 D4 D4 D4 C#4 C#4 C#4 C#4 D4 rest D4 D4 D4 D4 E4 E4 F#4 F#4 D4 D4 G4 G4 A4 G4 G4 G4 G4 F#4 F#4 F#4 F#4 G4 rest E4 E4 F4 F4 E4 E4 F4 F4 G4 G4 rest E4 E4 F4 F4 E4 E4 F4 F4 G4 G4 rest G4 G4 A4 A4 G4 G4 A4 A4 B4 B4 C5 C5 E4 E4 E4 E4 G4 G4 A4 A4 A4 G4 G4 rest C4 C4 D4 D4 C4 C4 F4 F4 E4 E4 D4 D4 C4 C4 rest F4 F4 E4 E4 D4 D4 C4 C4 C4 rest C4 C4 D4 D4 A3 A3 C4 C4 E4 E4 E4 E4 G4 rest E4 E4 F4 F4 E4 E4 F4 F4 G4 G4 rest E4 E4 F4 F4 E4 E4 F4 F4 G4 G4 rest G4 G4 A4 A4 G4 G4 A4 A4 B4 B4 C5 C5 E4 E4 E4 E4 G4 A4 A4 A4 G4 G4 rest C4 C4 D4 D4 C4 C4 F4 F4 E4 E4 D4 D4 C4 C4 rest F4 F4 E4 E4 D4 D4 C4 C4 C4 rest C4 C4 D4 D4 A3 A3 C4 C4 C4 C4 D4 D4 D4 C4 C4",
  "note_dur_seq": "0.29218 0.29218 0.289358 0.289358 0.180769 0.180769 0.23282 0.23282 0.278718 0.278718 0.461538 0.461538 0.169423 0.169423 0.522884 0.522884 0.23077 0.23077 0.200768 0.200768 0.30577 0.30577 0.302884 0.314424 0.182052 0.182052 0.309486 0.309486 0.212052 0.212052 0.234487 0.234487 0.230768 0.230768 0.461666 0.461666 0.245643 0.245643 0.335193 0.335193 0.301153 0.301153 0.188012 0.374488 0.374488 0.288462 0.343846 0.191731 0.191731 0.284806 0.284806 0.23077 0.23077 0.192499 0.192499 0.269039 0.269039 0.416538 0.416538 0.232178 0.232178 0.40013 0.40013 0.305769 0.305769 0.252503 0.252503 0.284036 0.284036 0.274038 0.393269 0.212822 0.212822 0.228718 0.228718 0.230768 0.230768 0.195063 0.195063 0.266091 0.266091 0.401922 0.401922 0.224103 0.224103 0.423205 0.423205 0.506541 0.506541 0.33577 0.33577 0.274038 0.373268 0.147947 0.147947 0.313591 0.313591 0.23077 0.23077 0.23077 0.23077 0.153522 0.153522 0.523783 0.523783 0.171156 0.171156 0.536154 0.536154 0.217754 0.217754 0.213785 0.213785 0.305767 0.305767 0.288462 0.358847 0.156601 0.156601 0.274938 0.274938 0.237692 0.237692 0.223846 0.223846 0.206024 0.206024 0.411284 0.411284 0.335768 0.335768 0.362692 0.362692 0.259232 0.259232 0.196152 0.42404 0.42404 0.23077 0.373272 0.169931 0.169931 0.224617 0.224617 0.297758 0.297758 0.200767 0.200767 0.202177 0.202177 0.520132 0.520132 0.172561 0.172561 0.519745 0.519745 0.174545 0.174545 0.246544 0.246544 0.316219 0.316219 0.274036 0.393269 0.201863 0.201863 0.224678 0.224678 0.245769 0.245769 0.230766 0.230766 0.159744 0.159744 0.472565 0.472565 0.255063 0.255063 0.381794 0.381794 0.450711 0.450711 0.430897 0.430897 0.317307 0.300961 0.201924 0.201924 0.277568 0.277568 0.22506 0.22506 0.202564 0.202564 0.212822 0.212822 0.495644 0.495644 0.177498 0.177498 0.454803 0.454803 0.490776 0.490776 0.219998 0.219998 0.41827 0.317951 0.146469 0.146469 0.281735 0.281735 0.261726 0.261726 0.215775 0.215775 0.187371 0.187371 0.519931 0.519931 0.18462 0.18462 0.477688 0.477688 0.38776 0.38776 0.307434 0.307434 0.389423 0.314999 0.17609 0.17609 0.300449 0.300449 0.200768 0.200768 0.260773 0.260773 0.13224 0.13224 0.488846 0.488846 0.299038 0.299038 0.404492 0.404492 0.260766 0.260766 0.23 0.23 0.234424 0.234424 0.46154 0.257881 0.173527 0.173527 0.305191 0.305191 0.12564 0.12564 0.288723 0.288723 0.243713 0.243713 0.191861 0.191861 0.271732 0.224613 0.224613 0.390129 0.390129 0.446541 0.446541 0.296795 0.296795 0.346153 0.372692 0.245769 0.245769 0.230766 0.230766 0.230774 0.230774 0.154037 0.154037 0.669808 0.669808 0.240002 0.320766 0.320766 0.23077 0.23077 0.230766 0.230766 0.154236 0.154236 0.669613 0.669613 0.330002 0.161407 0.161407 0.300125 0.300125 0.21558 0.21558 0.245964 0.245964 0.15699 0.15699 0.535316 0.535316 0.230766 0.230766 0.431541 0.431541 0.190384 0.190384 0.245193 0.375 0.375 0.418266 0.375 0.281475 0.206604 0.206604 0.213009 0.213009 0.18718 0.18718 0.507888 0.507888 0.202175 0.202175 0.416668 0.416668 0.523464 0.523464 0.272301 0.146031 0.146031 0.486289 0.486289 0.379032 0.379032 0.10878 0.30949 0.30949 0.14077 0.219741 0.219741 0.517562 0.517562 0.159752 0.159752 0.442558 0.442558 0.520958 0.520958 0.219811 0.219811 0.432693 0.301155 0.156395 0.156395 0.305146 0.305146 0.230766 0.230766 0.164474 0.164474 0.702637 0.702637 0.196739 0.320763 0.320763 0.200767 0.200767 0.260773 0.260773 0.164673 0.164673 0.702438 0.702438 0.226738 0.249802 0.249802 0.241729 0.241729 0.230767 0.230767 0.245781 0.245781 0.211663 0.211663 0.495644 0.495644 0.230766 0.230766 0.446533 0.446533 0.189812 0.211426 0.408768 0.408768 0.403847 0.302887 0.344414 0.215767 0.215767 0.192138 0.192138 0.217308 0.217308 0.513634 0.513634 0.20694 0.20694 0.40712 0.40712 0.527679 0.527679 0.197884 0.260773 0.260773 0.433334 0.433334 0.377235 0.377235 0.142502 0.218078 0.218078 0.153458 0.259491 0.259491 0.447822 0.447822 0.241669 0.241669 0.401345 0.401345 0.299096 0.299096 0.173416 0.374665 0.374665 0.47596 0.389427",
  "is_slur_seq": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1",
  "ph_dur": "0.165 0.12718 0.103589 0.185769 0.045 0.135769 0.095 0.13782 0.092949 0.185769 0.045 0.416538 0.045 0.124423 0.106346 0.416538 0.045 0.18577 0.045 0.155768 0.075001 0.230769 0.302884 0.314424 0.075 0.107052 0.123717 0.185769 0.045 0.167052 0.063717 0.17077 0.059999 0.170769 0.06 0.401666 0.059873 0.18577 0.045 0.290193 0.171346 0.129807 0.188012 0.143719 0.230769 0.288462 0.343846 0.059999 0.131732 0.099037 0.185769 0.045 0.18577 0.045 0.147499 0.083269 0.18577 0.045 0.371538 0.09 0.142178 0.088591 0.311539 0.15 0.155769 0.075001 0.177502 0.053268 0.230768 0.274038 0.393269 0.025002 0.18782 0.042948 0.18577 0.045 0.185768 0.045 0.150063 0.080708 0.185383 0.045385 0.356537 0.105001 0.119102 0.111668 0.311537 0.150002 0.356539 0.105 0.23077 0.274038 0.373268 0.045 0.102947 0.127823 0.185768 0.045 0.18577 0.045 0.18577 0.044998 0.108524 0.122246 0.401537 0.060001 0.111155 0.119616 0.416538 0.045 0.172754 0.058014 0.155771 0.074999 0.230768 0.288462 0.358847 0.045 0.111601 0.119167 0.155771 0.074999 0.162693 0.068077 0.155769 0.074999 0.131025 0.099745 0.311539 0.15 0.185768 0.045 0.317692 0.143847 0.115385 0.196152 0.150002 0.274038 0.23077 0.373272 0.044998 0.124933 0.105835 0.118782 0.111988 0.18577 0.044998 0.155769 0.075001 0.127176 0.103592 0.41654 0.044998 0.127563 0.103207 0.416538 0.044998 0.129547 0.101223 0.145321 0.085449 0.23077 0.274036 0.393269 0.025002 0.176861 0.053909 0.170769 0.060001 0.185768 0.044998 0.185768 0.045002 0.114742 0.116028 0.356537 0.105 0.150063 0.080708 0.301086 0.160454 0.290257 0.17128 0.259617 0.317307 0.300961 0.045193 0.156731 0.074039 0.203529 0.027241 0.197819 0.032951 0.169613 0.061153 0.151669 0.079102 0.416542 0.044998 0.1325 0.09827 0.356533 0.105003 0.385773 0.075768 0.14423 0.41827 0.317951 0.042625 0.103844 0.126923 0.154812 0.075958 0.185768 0.045002 0.170773 0.059998 0.127373 0.103397 0.416534 0.045002 0.139618 0.091152 0.386536 0.075001 0.312759 0.148781 0.158653 0.389423 0.314999 0.060001 0.116089 0.114681 0.185768 0.045002 0.155766 0.075001 0.185772 0.044998 0.087242 0.143528 0.345318 0.116219 0.182819 0.047951 0.356541 0.105 0.155766 0.075001 0.154999 0.075771 0.158653 0.46154 0.257881 0.045002 0.128525 0.102245 0.202946 0.027824 0.097816 0.132954 0.155769 0.074997 0.168716 0.062054 0.129807 0.271732 0.060001 0.164612 0.066154 0.323975 0.137566 0.308975 0.152565 0.14423 0.346153 0.372692 0.060001 0.185768 0.044998 0.185768 0.045002 0.185772 0.044998 0.109039 0.121731 0.548077 0.240002 0.134998 0.185768 0.045002 0.185768 0.044998 0.185768 0.045002 0.109234 0.121536 0.548077 0.330002 0.044998 0.116409 0.114361 0.185764 0.045002 0.170578 0.060192 0.185772 0.044998 0.111992 0.118778 0.416538 0.044998 0.185768 0.045002 0.386539 0.075001 0.115383 0.245193 0.10096 0.27404 0.418266 0.375 0.281475 0.035835 0.170769 0.060001 0.153008 0.077759 0.109421 0.121349 0.386539 0.075001 0.127174 0.103592 0.313076 0.148464 0.375 0.272301 0.045006 0.101025 0.129745 0.356544 0.104996 0.274036 0.10878 0.07872 0.23077 0.14077 0.089996 0.129745 0.101028 0.416534 0.044998 0.114754 0.11602 0.326538 0.135002 0.385956 0.075577 0.144234 0.432693 0.301155 0.044998 0.111397 0.11937 0.185776 0.044998 0.185768 0.044998 0.119476 0.11129 0.591347 0.196739 0.134995 0.185768 0.044998 0.155769 0.075005 0.185768 0.044998 0.119675 0.111099 0.591339 0.226738 0.104996 0.144806 0.08596 0.155769 0.075005 0.155762 0.075005 0.170776 0.059998 0.151665 0.079102 0.416542 0.044998 0.185768 0.044998 0.401535 0.189812 0.211426 0.120308 0.28846 0.403847 0.302887 0.344414 0.045006 0.170761 0.060005 0.132133 0.09864 0.118668 0.112099 0.401535 0.060005 0.146935 0.083832 0.323288 0.138252 0.389427 0.197884 0.104996 0.155777 0.074997 0.358337 0.103195 0.27404 0.142502 0.044998 0.17308 0.153458 0.135002 0.124489 0.106277 0.341545 0.119995 0.121674 0.109093 0.292252 0.169289 0.129807 0.173416 0.158318 0.216347 0.47596 0.389427",
  "input_type": "phoneme"
}
target = "/content/DiffSinger/infer_out/小手拉大手.wav"
ds.DiffSingerE2EInfer.example_run(c_xiaoshou, target=target)
Audio(filename=target)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
# 一次生成多个音频（AI 重录）
roll = {
  "text": "感 谢 我 们 一 起 走 了 那 么 久  SP 又 再 一 次 回 到  凉 凉 深 秋  SP",
  "ph_seq": "g an x ie w o m en y i q i z ou l e n a m e j iu iu SP y ou z ai y i c i h ui d ao ao l iang l iang sh en q iu iu SP",
  "note_seq": "E4 E4 E4 E4 D4 D4 C#4 C#4 E4 E4 E4 E4 D4 D4 D4 D4 D4 D4 C#4 C#4 C#4 C#4 D4 rest D4 D4 D4 D4 E4 E4 F#4 F#4 D4 D4 G4 G4 A4 G4 G4 G4 G4 F#4 F#4 F#4 F#4 G4 rest",
  "note_dur_seq": "0.176089 0.176089 0.300448 0.300448 0.20077 0.20077 0.260772 0.260772 0.132239 0.132239 0.488848 0.488848 0.299037 0.299037 0.40449 0.40449 0.260768 0.260768 0.229999 0.229999 0.234425 0.234425 0.461538 0.257883 0.173526 0.173526 0.30519 0.30519 0.12564 0.12564 0.288726 0.288726 0.243713 0.243713 0.191862 0.191862 0.271729 0.224616 0.224616 0.390127 0.390127 0.446539 0.446539 0.296796 0.296796 0.346154 1",
  "is_slur_seq": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0",
  "ph_dur": "0.060001 0.116088 0.114681 0.185767 0.045002 0.155768 0.075001 0.185771 0.044998 0.087241 0.143528 0.34532 0.116219 0.182818 0.047951 0.356539 0.105 0.155768 0.075001 0.154998 0.075771 0.158654 0.461538 0.257883 0.045002 0.128524 0.102245 0.202945 0.027824 0.097816 0.132954 0.155772 0.074997 0.168716 0.062054 0.129808 0.271729 0.060001 0.164615 0.066154 0.323973 0.137566 0.308973 0.152565 0.144231 0.346154 1",
  "input_type": "phoneme"
}
target_fmt = "/content/DiffSinger/infer_out/小手拉大手_take_{}.wav"
nums = 10
for i in range(1, nums + 1):
  ds.DiffSingerE2EInfer.example_run(
      roll,
      target=target_fmt.format(str(i).zfill(3))
  )



| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
47 47 47 47
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 62.88it/s]


| save audio: /content/DiffSinger/infer_out/小手拉大手_take_001.wav
| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
47 47 47 47
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 66.95it/s]


| save audio: /content/DiffSinger/infer_out/小手拉大手_take_002.wav
| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
47 47 47 47
Pass word-notes check.
===> gaussion start.


...

| save audio: /content/DiffSinger/infer_out/小手拉大手_take_008.wav
| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
47 47 47 47
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 66.62it/s]


| save audio: /content/DiffSinger/infer_out/小手拉大手_take_009.wav
| load 'model' from 'checkpoints/0228_opencpop_ds100_rel/model_ckpt_steps_160000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| load HifiGAN:  checkpoints/0109_hifigan_bigpopcs_hop128/model_ckpt_steps_280000.ckpt
Removing weight norm...
47 47 47 47
Pass word-notes check.
===> gaussion start.


sample time step: 100%|██████████| 100/100 [00:01<00:00, 66.54it/s]


| save audio: /content/DiffSinger/infer_out/小手拉大手_take_010.wav
